# Diabetes-  Aprendizaje

## Preparacion de Datos

### Importar Librerias

In [224]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt  
import seaborn as sns 
import ipywidgets as widgets
from sklearn.cluster import KMeans

### Importar archivo csv

In [225]:
diabetes = pd.read_csv("diabetes.csv")

### Informacion Variables

In [226]:
nVariables = diabetes.columns.values
porcentajePeligroMedia = 30;
names = []
cualcuant = []
tipo = []
media = []
moda = []
mediana = []
desviacionEstandar = []
advertencias = []
indexpd = []
tipoD = []
minValue = []
maxValue = []
rango = []
Significado = []
c = 0

for i in nVariables:
    c = c+1;
    indexpd.append(c)
    if diabetes[i].dtype == "int64" or diabetes[i].dtype == "float64":
        names.append(i)
        tipoD.append(diabetes[i].dtype)
        minValue.append(diabetes[i].min())
        maxValue.append(diabetes[i].max())
        minValueDat = diabetes[i].min()
        maxValueDat = diabetes[i].max()
        media.append(diabetes[i].mean())
        moda.append(diabetes[i].mode(dropna=True)[0])
        mediana.append(diabetes[i].median())
        desviacionEstandar.append(diabetes[i].std())
        desviacionEstandarDat = diabetes[i].std()
        mediaDat = diabetes[i].mean()
        if minValueDat==0 and maxValueDat==1 and diabetes[i].dtype == "int64":
            cualcuant.append("Cualitativa")
            tipo.append("Ordinal")
            advertencias.append("Cualitativa, medidas de tendencia no muy utiles")
        else:
            cualcuant.append("Cuantitativa")
            if desviacionEstandarDat >= mediaDat*porcentajePeligroMedia/100:
                Advertencia = "La desviacion estandar es mayor a {}% la media".format(porcentajePeligroMedia)
            else:
                Advertencia = "Ninguna"
            advertencias.append(Advertencia)
            if diabetes[i].dtype == "int64":
                tipo.append("Discreta")
            else:
                tipo.append("Continua")
        rango.append("[{},{}]".format(minValueDat,maxValueDat))
    else:
        media.append("NaN")
        moda.append("NaN")
        mediana.append("NaN")
        desviacionEstandar.append("NaN")
        names.append(i)
        cualcuant.append("Cualitativa")
        tipo.append("Nominal")
        advertencias.append("Ninguna")
        rango.append("NaN")
Significado.append("Numero de Veces que ha estado en embarazo")
Significado.append("Concentracion de plasma de glucosa en 2 horas en examen oral de glucosa")
Significado.append("Presion diastolica de la sangre (mmHg)")
Significado.append("Grosor de la piel en el tricep")
Significado.append("2 horas con suero de insulina (muU/ml)")
Significado.append("Indice de masa corporal")
Significado.append("Funcion del arbol genealogico de diabetes")
Significado.append("Edad en años")
Significado.append("Variable de clase 268 de 768 son 1, otros son 0")
diabetesExp = {"Significado":pd.Series(Significado,index=names),"Tipo":pd.Series(tipoD,index=names),"Cual/Cuan":pd.Series(cualcuant,index=names),"Subtipo":pd.Series(tipo,index=names),"Rango":pd.Series(rango,index=names),"Media":pd.Series(media,index=names),"Moda":pd.Series(moda,index=names),"Mediana":pd.Series(mediana,index=names),"Desviacion Estandar":pd.Series(desviacionEstandar,index=names),"Advertencia":pd.Series(advertencias,index=names)}
diabetesDat = pd.DataFrame(diabetesExp)
diabetesDat

,Significado,Tipo,Cual/Cuan,Subtipo,Rango,Media,Moda,Mediana,Desviacion Estandar,Advertencia
Pregnancies,Numero de Veces que ha estado en embarazo,int64,Cuantitativa,Discreta,"[0,17]",3.845052,1.000,3.0000,3.369578,La desviacion estandar es mayor a 30% la media
Glucose,Concentracion de plasma de glucosa en 2 horas ...,int64,Cuantitativa,Discreta,"[0,199]",120.894531,99.000,117.0000,31.972618,Ninguna
BloodPressure,Presion diastolica de la sangre (mmHg),int64,Cuantitativa,Discreta,"[0,122]",69.105469,70.000,72.0000,19.355807,Ninguna
SkinThickness,Grosor de la piel en el tricep,int64,Cuantitativa,Discreta,"[0,99]",20.536458,0.000,23.0000,15.952218,La desviacion estandar es mayor a 30% la media
Insulin,2 horas con suero de insulina (muU/ml),int64,Cuantitativa,Discreta,"[0,846]",79.799479,0.000,30.5000,115.244002,La desviacion estandar es mayor a 30% la media
BMI,Indice de masa corporal,float64,Cuantitativa,Continua,"[0.0,67.1]",31.992578,32.000,32.0000,7.884160,Ninguna
DiabetesPedigreeFunction,Funcion del arbol genealogico de diabetes,float64,Cuantitativa,Continua,"[0.078,2.42]",0.471876,0.254,0.3725,0.331329,La desviacion estandar es mayor a 30% la media
Age,Edad en años,int64,Cuantitativa,Discreta,"[21,81]",33.240885,22.000,29.0000,11.760232,La desviacion estandar es mayor a 30% la media
Outcome,"Variable de clase 268 de 768 son 1, otros son 0",int64,Cualitativa,Ordinal,"[0,1]",0.348958,0.000,0.0000,0.476951,"Cualitativa, medidas de tendencia no muy utiles"


### Cambiar Ceros no probables por la mediana

In [227]:
diabetes["BMI"] = diabetes["BMI"].replace(0,diabetes["BMI"].median())
diabetes["BloodPressure"] = diabetes["BloodPressure"].replace(0,diabetes["BloodPressure"].median())

### Ordenar Datos

In [228]:
diabetes = diabetes.sort_values("Outcome")

### Limpiar Datos no Utiles y Outcome

In [229]:
nVariables = diabetes.columns.values
checks = []
print("Valores a eliminar: ")
for n in range(len(nVariables)):
    checks.append(widgets.Checkbox(value=False,description=nVariables[n],disabled=False))
    display(checks[n])
#Activamos por defecto las que creemos que no afectan asi como el outcome
#Outcome
if checks[len(checks)-1].description == "Outcome":
    checks[len(checks)-1].value = True
    checks[len(checks)-1].disabled = True

Valores a eliminar: 


Checkbox(value=False, description='Pregnancies')

Checkbox(value=False, description='Glucose')

Checkbox(value=False, description='BloodPressure')

Checkbox(value=False, description='SkinThickness')

Checkbox(value=False, description='Insulin')

Checkbox(value=False, description='BMI')

Checkbox(value=False, description='DiabetesPedigreeFunction')

Checkbox(value=False, description='Age')

Checkbox(value=False, description='Outcome')

In [230]:
eliminar = [];
for n in range(len(checks)):
    if checks[n].value:
        eliminar.append(checks[n].description)
if eliminar != []:
    diabetes = diabetes.drop(columns=eliminar)

### Unir por Rango

In [158]:
# Listo por si se necesita
"""
n = "DiabetesPedigreeFunction"
Rango = [[diabetes[n].min(),diabetes[n].quantile(0.25)],[diabetes[n].quantile(0.25),diabetes[n].quantile(0.5)],[diabetes[n].quantile(0.5),diabetes[n].quantile(0.75)],[diabetes[n].quantile(0.75),diabetes[n].max()]]
diabetes[n] = np.where(diabetes[n].between(Rango[0][0],Rango[0][1]), 0, diabetes[n])
diabetes[n] = np.where(diabetes[n].between(Rango[1][0],Rango[1][1]), 1, diabetes[n])
diabetes[n] = np.where(diabetes[n].between(Rango[2][0],Rango[2][1]), 2, diabetes[n])
diabetes[n] = np.where(diabetes[n].between(Rango[3][0],Rango[3][1]), 3, diabetes[n])

n = "Glucose"
Rango = [[diabetes[n].min(),diabetes[n].quantile(0.25)],[diabetes[n].quantile(0.25),diabetes[n].quantile(0.5)],[diabetes[n].quantile(0.5),diabetes[n].quantile(0.75)],[diabetes[n].quantile(0.75),diabetes[n].max()]]
diabetes[n] = np.where(diabetes[n].between(Rango[0][0],Rango[0][1]), 0, diabetes[n])
diabetes[n] = np.where(diabetes[n].between(Rango[1][0],Rango[1][1]), 1, diabetes[n])
diabetes[n] = np.where(diabetes[n].between(Rango[2][0],Rango[2][1]), 2, diabetes[n])
diabetes[n] = np.where(diabetes[n].between(Rango[3][0],Rango[3][1]), 3, diabetes[n])

n = "Insulin"
Rango = [[diabetes[n].min(),diabetes[n].quantile(0.25)],[diabetes[n].quantile(0.25),diabetes[n].quantile(0.5)],[diabetes[n].quantile(0.5),diabetes[n].quantile(0.75)],[diabetes[n].quantile(0.75),diabetes[n].max()]]
diabetes[n] = np.where(diabetes[n].between(Rango[0][0],Rango[0][1]), 0, diabetes[n])
diabetes[n] = np.where(diabetes[n].between(Rango[1][0],Rango[1][1]), 1, diabetes[n])
diabetes[n] = np.where(diabetes[n].between(Rango[2][0],Rango[2][1]), 2, diabetes[n])
diabetes[n] = np.where(diabetes[n].between(Rango[3][0],Rango[3][1]), 3, diabetes[n])
""""

### Rellenar Espacios Vacios

In [231]:
nVariables = diabetes.columns.values
for n in nVariables:
    diabetes[n] = diabetes[n].fillna(diabetes[n].median())

## Implementacion K-means

### Asignar K

In [232]:
k = 2

### Determinar los centros

In [233]:
kmeans = KMeans(n_clusters=k, random_state=0).fit(diabetes)

### Mostrar los Centros

In [234]:
print(kmeans.cluster_centers_)

[[  3.7030303  141.46060606  72.78787879  31.2        253.70909091
   34.98545455   0.59724848  33.7030303 ]
 [  3.88391376 115.26699834  72.27694859  17.6185738   32.21227197
   31.75737977   0.43757048  33.11442786]]


### Mostrar los Labels

In [235]:
print(kmeans.labels_)

[1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0
 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 0
 1 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 0 1 1 1 0 0
 1 0 1 1 1 0 1 1 0 1 1 1 0 1 1 0 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 0 1 0 1 0 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 0 1 1 0 1 0 0
 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 0 1 1 1
 1 1 1 1 1 0 0 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1
 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 0 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 0 1 1 1
 1 0 0 1 1 0 1 1 1 1 0 1 

### Prediccion

In [117]:
#Parte extra que se iba a tener para ingresar datos y asi identificar si se tiene o no diabetes.
nVariables = diabetes.columns.values
minV = 0
maxV = 10
stepV = 0.1
valuesPred = []
for n in range(len(nVariables)):
    valuesPred.append(widgets.BoundedFloatText(value=0,min=minV,max=maxV,step=stepV,description=nVariables[n]+": ",disabled=False))
    display(valuesPred[n])

BoundedFloatText(value=0.0, description='Pregnancies: ', max=10.0, step=0.1)

BoundedFloatText(value=0.0, description='Glucose: ', max=10.0, step=0.1)

BoundedFloatText(value=0.0, description='BloodPressure: ', max=10.0, step=0.1)

BoundedFloatText(value=0.0, description='SkinThickness: ', max=10.0, step=0.1)

BoundedFloatText(value=0.0, description='Insulin: ', max=10.0, step=0.1)

BoundedFloatText(value=0.0, description='BMI: ', max=10.0, step=0.1)

BoundedFloatText(value=0.0, description='DiabetesPedigreeFunction: ', max=10.0, step=0.1)

BoundedFloatText(value=0.0, description='Age: ', max=10.0, step=0.1)

In [178]:
"""
vPred = [];
for n in range(len(nVariables)):
    vPred.append(valuesPred[n].value)
print(kmeans.predict(vPred))
"""

'\nvPred = [];\nfor n in range(len(nVariables)):\n    vPred.append(valuesPred[n].value)\nprint(kmeans.predict(vPred))\n'

# Preguntas

## ¿Crees que estos centros puedan ser representativos de los datos? ¿Por qué?

Si, pueden ser repersentativos, solamente que algunos datos los cuales no son muy exactos o tienen valores no probables, pueden estar modificando los resultados de los centros, por lo que pueden ser no muy exactos, esto se podria mejorar usando rangos para algunos datos.

## ¿Cómo obtuviste el valor de k a usar?

Debido a que solo tenemos 2 posibles resultados, si tiene o no tiene diabetes.

## ¿Los centros serían más representativos si usaras un valor más alto? ¿Más bajo?

Si usaramos mas datos es probable que obtendriamos mejores resultados, si tuvieramos menos tendriamos un peor resultado, y podriamos mejorar el resultado usando rangos, solamente que por tiempo no se ha terminado esa parte.

## ¿Qué distancia tienen los centros entre sí? ¿Hay alguno que este muy cercano a otros?

In [236]:
distancia = (kmeans.cluster_centers_[0]-kmeans.cluster_centers_[1])**2;
distancia = sum(distancia)**(1/2)
print(distancia)

223.47815823205042


La distancia entre los centrso es de 223.4781, como solo tenemos 2 centros solo tenemos 1 distancia entre ellos.

## ¿Qué pasaría con los centros si tuviéramos muchos outliers en el análisis de cajas y bigotes?

Si tuvieramos muchos outliers, tendriamos peores resultados, y unos centros muy desubicados, los cuales no nos servirian.

## ¿Qué puedes decir de los datos basándose en los centros?

Segun los centros actuales es mucho mas probable que si se tenga diabetes a que no se tenga (Tenemos mas valores en el centro 1)